# **Eight Puzzle**

---






1. From a random start state                   
2. Generate goal state
3. Check if solvable
4. IF solvable:
      *   Using at least 2 different heuristic functions
      *   Provide estimate of algorithms complexity
      *   Measure memory effort (number of nodes expanded) and run time for each random state and each heuristic
5. Provide simple GUI
(Menu)



---


Team Members: Zeynep Avci, Sara Khazna and Nur Sarhan

---



Changes made for the correction:

From the old code, the unused bits were removed, and the puzzle is now generated as a normal array.

*   def get_legal_moves + def generate_move were updated from the team members

*   the solvability was incorrect at some random generated puzzles -> def is_puzzle_solvable was written completly by members (is easy, but effective compared to the old one) 

*   def solve_a_star was corrected and adapted to new structure -> two seperate functions got fusin into one

*   heuristics were too complicated to understand, so the members wrote the functions again after understanding them

*   Menu stayed almost the same, for random generated puzzles we now use np.random.choice (because the method used before was complicated); adjustments were made(e.g. function names)






Run code from top to bottom, (Run all)

In [ ]:
import heapq


# ----------------------------------------------------------------------------------------------------------------------
# Class for defining data structure (heap), along with important methods
# ----------------------------------------------------------------------------------------------------------------------
class PriorityList:
    
    def __init__(self, priority_func):
        self.priority_func = priority_func
        self.heap = []
        
    # add a new item on the heap   
    def add(self, item):
        heapq.heappush(self.heap, (self.priority_func(item), item))

    # pop and return the smallest item from the heap 
    def remove(self):
        (_, item) = heapq.heappop(self.heap)
        return item

     # define empty heap
    def empty(self):
        return len(self.heap) == 0


# define goal state of the puzzle 
goal_state = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]



# ----------------------------------------------------------------------------------------------------------------------
# Class EightPuzzle
# ----------------------------------------------------------------------------------------------------------------------
class EightPuzzle:
    # The puzzle is visualized as an array that consists of 9 elements, where 0 corresponds to the empty value
    def __init__(self, array):
        self.array = list(array)
        for i in range(len(array)):
            if self.array[i] == 0:
                self.empty_value = i
                break      

  
    def get_start_state(self):       # state consists of ([array], empty_value, [path])
        return (self.array, self.empty_value, [])

    def is_goal_state_reached(self, state):
        # define state of puzzle 
        array, empty_value, _ = state
        
        # loop through the array and check if items are ordered correctly
        for i in range(8):
            # if not return false
            if array[i] != i+1:
                return False
        # if true assign the 0 to the last item in the array        
        return empty_value == 8

   
    # ----------------------------------------------------------------------------------------------------------------------
    # def get_legal_moves(self, state):
    # Method to get back possible moves to solve puzzle and reach goal state
    # ----------------------------------------------------------------------------------------------------------------------
    def get_legal_moves(self, state):
        
        # initialize array to store possible moves of the chosen path
        moves = []

        array, empty_value, path = state            

        
        def generate_move(incr, action):
            # store path
            path_copy = list(path)
            # action is added to path copy
            path_copy.append(action)
            arr_copy = list(array)
            # switch the position of the empty value
            arr_copy[empty_value], arr_copy[empty_value + incr] = arr_copy[empty_value + incr], arr_copy[empty_value]
            # return legal moves 
            moves.append((arr_copy, empty_value + incr, path_copy))

        # 3, 1, 2, 
        # 5, 4, 7,
        # 6 ,0, 8
      
       
        if empty_value >= 3:        # move empty value to the top, if it is position is >= 3 in the list, go 3 positions back in the array
            generate_move(-3, 'T')
       
        if empty_value <= 5:        # move empty value to the bottom, if it is position is <= 5 in the list, go 3 positions further in the array
            generate_move(3, 'B')
        
        if (empty_value % 3) > 0:   # move empty value to the left, if the modulo rest is > 0, move one to the left in the array
            generate_move(-1, 'L')
       
        if (empty_value % 3) < 2:   # move empty value to the right, if if the modulo rest is < 2, move one to the right in the array   
            generate_move(1, 'R')    

        return moves


# ----------------------------------------------------------------------------------------------------------------------
# def is_puzzle_solvable(arr):
# This method calculates the inversion count of a given puzzle (checks elements that are not in their right position)
# ----------------------------------------------------------------------------------------------------------------------

def is_puzzle_solvable(arr):
     # initialize inversion count to calculate and find out whether puzzle is solvable or not
     inv_count = 0
     # define empty value in the list
     zero_value = 0

     for i in range(0,9):
          for j in range(i + 1, 9):
            # check if any element in the array is a 0 and the element's position value
            if arr[j] != zero_value and arr[i] != zero_value and arr[i] > arr[j]:
                   # increase inversion count by 1
                   inv_count += 1

     # return if inversion count is even.
     return inv_count % 2 == 0 




# A * Algorithm and Manhattan, Hamming Heuristics

# ------------------------------------------------------------------------------------------------------------------
# def: solve_a_star(puzzle, heuristic)
# This method is for solving the puzzle according to the a * algorithm
# ------------------------------------------------------------------------------------------------------------------

def solve_a_star(puzzle, heuristic):
  # A* uses the path cost from start state and the heuristic estimate to a goal
    totalCost = lambda a_start: len(a_start[2]) + heuristic(a_start)
    method = PriorityList(totalCost)
    start = puzzle.get_start_state()  # get start state of puzzle
    explored_nodes = set([str(start[0])]) # store array in a set
    method.add(start) # pass the start state to the used method
    
    while not method.empty():
        node = method.remove()
        if puzzle.is_goal_state_reached(node):
            # return the solution path and no. of explored nodes
            solv = (node[2], len(explored_nodes))
            return solv

            #if not reached goal state yet, then move with legal moves
        for move in puzzle.get_legal_moves(node):
            arr_copy = str(move[0])
            if arr_copy not in explored_nodes:                
                explored_nodes.add(arr_copy)
                method.add(move)  # pass the moves to the used method
    return None

# ------------------------------------------------------------------------------------------------------------------
# def manhattan_h(array):
# This method is to calculate The Manhattan distance, which is the sum of the horizontal and vertical distances. 
# The goal distance estimates the sum of the Manhattan distances from the tiles to their goal positions
# ------------------------------------------------------------------------------------------------------------------
def manhattan_h(array):
       distance = 0
     
       for i in range (len(array)):
              #target row - current row
              target_row = int((array[i]-1) / 3)
              current_row = int(i / 3)
              #target column - current column
              target_col = int((array[i]-1) % 3)
              current_col = int(i % 3) 
              # formula to calculate manhattan distance
              distance += abs(current_row - target_row) + abs(current_col - target_col)
       return distance    

# ------------------------------------------------------------------------------------------------------------------
# def hamming_h(array):
# This method is to calculate the number of misplaced tiles (elements),
# that are currently not in the right position.
# ------------------------------------------------------------------------------------------------------------------
def hamming_h(array):
     # initialize variable to count hamming distance 
     misplaced_tiles = 0
     # for loop to iterate through puzzle array
     for i in range(len(array)):
         if (array[i] != 0 and array[i] != i+1):  # if the number in position i=0, or lies in the false position
             misplaced_tiles +=1                  # increase hamming distance and return the number of misplaced tiles
     return misplaced_tiles




# ------------------------------------------------------------------------------------------------------------------
#
#
# ------------------------------------------------------------------------------------------------------------------     



---


main.py


---





In [ ]:

import time
import numpy as np


# ---------------------------------------------------------------
# Defines menu that is printed for the user to choose from
# ---------------------------------------------------------------


def menu():
    print("---------------------------------------------------------------------------------")
    print("Eight Puzzle Implementation using Heuristic Search: ")
    print("---------------------------------------------------------------------------------")
    print("Choose a number from 1 to 6:")
    print("1. Display Random Puzzle and check solvability: ")
    print("2. Display Goal State ")
    print("3. Solve the puzzle using the Manhattan Heuristic:")
    print("4. Solve the puzzle using the Hamming Heuristic:")
    print("5. Print the Results of both Heuristics:  ")
    print("6. Quit. ")
    print("---------------------------------------------------------------------------------")


def main():
    
    start_state = np.random.choice(np.arange(0, 9), replace=False, size=(9))
    puzzle = EightPuzzle(start_state)

    def print_random_puzzle_and_check_solvability():      
        board_sqr = start_state.reshape(3,3)
        show_board = str(board_sqr).replace('[', ' ').replace(']', '')
        
        print(show_board)
      
        if is_puzzle_solvable(start_state):
            print("This puzzle is 'SOLVABLE'")
        else:
            print("This puzzle is 'NOT SOLVABLE', Please RESTART the program, by pressing:'6'")
            
         
    def solve_puzzle_with_manhattan():
        
        print("---------------------------------------------------------------------------------")
        #path, count = p.solve_a_star(manhattan_h)
        print ("Manhattan Distance: ", manhattan_h(start_state))
        start_time = time.time()
        solution = solve_a_star(puzzle, lambda man_state: manhattan_h(man_state[0]))
        print("--- %s milliseconds for MANHATTAN ---" % ((time.time() - start_time) * 1000))
        print("Solved with Manhattan distance exploring", solution[1], "nodes")
        print("---------------------------------------------------------------------------------")
        


    def solve_puzzle_with_hamming():
        
        print("---------------------------------------------------------------------------------")
        #path, count = p.solve_a_star(hamming_h)
        print ("Hamming Distance: ", hamming_h(start_state))
        start_time = time.time()
        solution = solve_a_star(puzzle, lambda ham_state: hamming_h(ham_state[0]))
        print("--- %s milliseconds for HAMMING ---" % ((time.time() - start_time) * 1000))
        print("Solved with Hamming distance exploring", solution[1], "nodes")
        print("---------------------------------------------------------------------------------")
        

    def compare_results():
        print("---------------------------------------------------------------------------------")
        # Measure Manhattan Complexity:
        
        
        print ("Manhattan Distance: ", manhattan_h(start_state))
        start_time = time.time()
        solution = solve_a_star(puzzle, lambda man_state: manhattan_h(man_state[0]))
        print("--- %s milliseconds for MANHATTAN ---" % ((time.time() - start_time) * 1000))
        print("Solved with Manhattan distance exploring", solution[1], "nodes")
        print("Path length:", len(solution[0]))
        print("---------------------------------------------------------------------------------")

        # Measure Hamming Complexity:

        print ("Hamming Distance: ", hamming_h(start_state))
        start_time = time.time()
        solution = solve_a_star(puzzle, lambda ham_state: hamming_h(ham_state[0]))
        print("--- %s milliseconds for HAMMING ---" % ((time.time() - start_time) * 1000))
        print("Solved with Hamming distance exploring", solution[1], "nodes")
        print("Path length:", len(solution[0]))
        print("---------------------------------------------------------------------------------")

        

    menu()

    choice = (input("Enter an integer between 1-6: "))

    run = True
    while run:
        if choice == "1":
            print_random_puzzle_and_check_solvability()
            menu()
            choice = (input("Enter an integer between 1-6: "))
        elif choice == "2":
            print("Goal State:" , '\n')
            print(goal_state)
            menu()
            choice = (input("Enter an integer between 1-6: "))
        elif choice == "3":
            print("Solved with Manhattan:")
            solve_puzzle_with_manhattan()
            menu()
            choice = (input("Enter an integer between 1-6: "))
        elif choice == "4":
            print("Solved with Hamming:")
            solve_puzzle_with_hamming()
            menu()
            choice = (input("Enter an integer between 1-6: "))
        elif choice == "5":
            compare_results()
            menu()
            choice = (input("Enter an integer between 1-6: "))
        elif choice == "6":
            print("Quit Puzzle")
            run = False
        else:
            print("Invalid Option! ")
            choice = (input("Enter an integer between 1-6: "))


if __name__ == "__main__":
    main()


---------------------------------------------------------------------------------
Eight Puzzle Implementation using Heuristic Search: 
---------------------------------------------------------------------------------
Choose a number from 1 to 6:
1. Display Random Puzzle and check solvability: 
2. Display Goal State 
3. Solve the puzzle using the Manhattan Heuristic:
4. Solve the puzzle using the Hamming Heuristic:
5. Print the Results of both Heuristics:  
6. Quit. 
---------------------------------------------------------------------------------
Enter an integer between 1-6: 1
  3 2 8
  1 0 5
  4 7 6
This puzzle is 'SOLVABLE'
---------------------------------------------------------------------------------
Eight Puzzle Implementation using Heuristic Search: 
---------------------------------------------------------------------------------
Choose a number from 1 to 6:
1. Display Random Puzzle and check solvability: 
2. Display Goal State 
3. Solve the puzzle using the Manhattan Heurist